## Incompressible liquids
### Main Tank


In [39]:
using ModelingToolkit, Plots, DifferentialEquations, Revise, ModelingToolkitStandardLibrary, Unitful
using Symbolics

Sym = Symbolics

@variables temps
@register_symbolic cpfunc(temps)
@register_symbolic kfunc(temps)

@connector function FlowPin(; name)
    sts = @variables P=1.0 ṁ=1.0
    ps = @parameters cp k
    NonlinearSystem(Equation[],  sts, []; name = name)
end

@connector function HeatPin(; name)
    sts = @variables T=300 Q̇=0.0 [connect = Flow]
    NonlinearSystem(Equation[],  sts, []; name = name)
end

@connector function WorkPin(; name)
    sts = @variables Ẇ = 0.0
    NonlinearSystem(Equation[], sts, []; name = name)
end

# convention = +Q = Qin
@connector function HeatFlowPin(; name, Qin = 0.0)
    sts = @variables Q̇=Qin [input = true]
    ps = @parameters Qin = Qin
    eqs=[
        Q̇ ~ Qin
    ]
    NonlinearSystem(eqs, sts, ps; name = name)
end

# convention = +W = Win
@connector function WorkPort(; name)
    sts = @variables Ẇ = 0.0 [output = true]
    NonlinearSystem(Equation[],  sts, []; name = name)
end

@connector function ThermoPin(; name)
    sts = @variables P=1.0 ṁ=1.0 T=293.15 cp k
    # ps = @parameters cp k
    eq = [
        cp ~ cpfunc(T)
        k ~ kfunc(T)
    ]
    NonlinearSystem(eq,  sts, []; name = name)
end

@component function FlowSource(;name, mflow = 1.0, P = 10, T = 300)
    @named src = ThermoPin()
    ps = @parameters mflow=mflow P=P T=T
    # 0 variables, 3 equations for pin
    eqs = [
        src.T ~ T
        src.P ~ P
        src.ṁ ~ mflow
    ]
    compose(NonlinearSystem(eqs,  [], ps; name = name),src)
end

@component function ThermoGround(; name)
    @named gnd = ThermoPin()
    eqs = [gnd.P ~ 0;   gnd.T ~ 273.15]
    compose(NonlinearSystem(eqs,  [], []; name = name), gnd)
end

@component function ThermoHeat(; name)
    @named p = ThermoPin()
    @named n = ThermoPin()
    @named h = HeatFlowPin()
    #   0 variables 3 equations for pin
    eqs = [
        p.ṁ ~ n.ṁ                               # conservation of mass
        n.T ~ p.T + h.Q̇/(p.ṁ*p.cp)
        n.P ~ p.P
    ]
    compose(NonlinearSystem(eqs,[],[]; name = name), p,n,h)
end

@component function ThermoCompressor(; name, η = 1.0, rp = 3.5)
    @named p = ThermoPin()
    @named n = ThermoPin()
    @named w = WorkPin()

    ps = @parameters rp = rp η = η
    eqs = [
        p.ṁ ~ n.ṁ                               # conservation of mass
        n.P ~ p.P * rp
        n.T ~ p.T * (η + rp^((p.k-1)/p.k)- η*(rp^((p.k-1)/p.k))) / (rp^((p.k-1)/p.k))
        w.Ẇ ~ p.ṁ * p.cp * (n.T - p.T)
    ]
    compose(NonlinearSystem(eqs,[],ps; name = name), p,n,w)
end

@component function ThermoTurbine(; name, η = 1.0, rp = 3.5)
    @named p = ThermoPin()
    @named n = ThermoPin()
    @named w = WorkPin()

    ps = @parameters rp = rp η = η
    eqs = [
        p.ṁ ~ n.ṁ                               # conservation of mass
        n.P ~ p.P * rp
        n.T ~ p.T * ((1.0 - η - (rp^((p.k-1)/p.k))) / (-η))
        w.Ẇ ~ p.ṁ * p.cp * (n.T - p.T)
    ]
    compose(NonlinearSystem(eqs,[],ps; name = name), p,n,w)
end

mf = FlowSource(name = :mf);
cc = ThermoCompressor(name = :cc)
qq = ThermoHeat(name = :qq)
tt = ThermoTurbine(name = :tt)

con = [connect(mf.src,cc.p)
        connect(cc.n,qq.p)
        connect(qq.n,tt.p)]

        
@named connected_sys =  NonlinearSystem(con,[],[]; systems = [mf, cc, qq, tt]);
# updaded_eq = substitute(equations(expand_connections(connected_sys)))
@named newsys = NonlinearSystem(equations(expand_connections(connected_sys)),states(connected_sys),parameters(connected_sys))
simple_sys = alias_elimination(newsys)
parameters(simple_sys)
# states(simple_sys)
# showeq(simple_sys)

┌ Warning: src contains 0 flow variables, yet 5 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40
┌ Warning: p contains 0 flow variables, yet 5 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40
┌ Warning: n contains 0 flow variables, yet 5 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia

┌ Warning: p contains 0 flow variables, yet 5 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40
┌ Warning: n contains 0 flow variables, yet 5 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40


┌ Warning: p contains 0 flow variables, yet 5 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40
┌ Warning: n contains 0 flow variables, yet 5 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40
┌ Warning: w contains 0 flow variables, yet 1 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\p

8-element Vector{SymbolicUtils.BasicSymbolic{Real}}:
 mf₊mflow
 mf₊P
 mf₊T
 cc₊rp
 cc₊η
 qq₊h₊Qin
 tt₊rp
 tt₊η

In [27]:
# states(newsys)
# ModelingToolkit.get_u0_p(newsys)
parameters(qq)

5-element Vector{Any}:
 p₊cp
 p₊k
 n₊cp
 n₊k
 h₊Qin

In [20]:
parameters(newsys)

22-element Vector{SymbolicUtils.BasicSymbolic{Real}}:
 mf₊mflow
 mf₊P
 mf₊T
 mf₊src₊cp(T)
 mf₊src₊k(T)
 cc₊rp
 cc₊η
 cc₊p₊cp(T)
 cc₊p₊k(T)
 cc₊n₊cp(T)
 ⋮
 qq₊n₊cp(T)
 qq₊n₊k(T)
 qq₊h₊Qin
 tt₊rp
 tt₊η
 tt₊p₊cp(T)
 tt₊p₊k(T)
 tt₊n₊cp(T)
 tt₊n₊k(T)

In [ ]:
parameters(newsys)



# p = [mf.mflow => 100
#     mf.P => 10
#     mf.T => 300
#     cc.rp => 3.5
#     cc.η => .9
#     qq.h.Qin => 10e6
#     tt.rp => 3.5
#     tt.η => 0.9]

showeq(x) = equations(expand_connections(x))
eq = showeq(newsys)

# get_variables(newsys)
# Sym.solve_for(eq,states(newsys))
# Symbolics.build_function(equations(newsys),parameters(newsys))
# simple_sys = structural_simplify(newsys)
# showeq(newsys)
# simplify(newsys)
# # full_equations(simple_sys)
# prob = NonlinearProblem(newsys,[],p)
# # solve(simple_sys)
# NonlinearSolve(prob)
# solve(prob)
# states(newsys)

# para = parameters(newsys)
# stat = states(newsys)

# intersect(para,stat)
# display(para)
# display(stat)
